# Spectral Extraction Tutorials

In [21]:
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.table import Table
from astropy.io import fits
from astropy.coordinates import SkyCoord
import shutil
import warnings
warnings.filterwarnings("ignore")

In [2]:
cat = Table.read('../Data/eROSITA_Catalogues.fit')

In [3]:
is_spurious = cat['FLAG_SP_SNR'].astype(bool)
cat = cat[~is_spurious]

cat_coord = SkyCoord(cat['RA'], cat['DEC'], unit='deg')

coord_obj = SkyCoord(83.63240, 22.01740, unit='deg')

distance = cat_coord.separation(coord_obj)

In [4]:
Image_file = '../Data/Images/merged_exp_corr_200_2300.fits'

In [5]:
from astropy.wcs import WCS

# Read the image file header
with fits.open(Image_file) as hdul:
    header = hdul[0].header

# Extract the WCS information from the header
wcs = WCS(header)

# Calculate the extent of the image
naxis1 = header['NAXIS1']
naxis2 = header['NAXIS2']
ra_min, dec_min = wcs.wcs_pix2world([[0, 0]], 0)[0]
ra_max, dec_max = wcs.wcs_pix2world([[naxis1, naxis2]], 0)[0]

# Define the extent of the image
extent = SkyCoord([ra_min, ra_max], [dec_min, dec_max], unit='deg')
extent

<SkyCoord (ICRS): (ra, dec) in deg
    [(85.55021296, 20.20636293), (81.66400501, 23.80665032)]>

In [6]:
# Apply cuts on the catalog using ra_min, ra_max, dec_min, dec_max
cat_cut = cat[(cat['RA'] >= ra_max) & (cat['RA'] <= ra_min) & 
              (cat['DEC'] >= dec_min) & (cat['DEC'] <= dec_max)]

with open('../Data/Spectrum/cat_deg.reg', 'w') as f:
    # f.write('fk5\n')
    for row in cat_cut:
        f.write(f'circle({row["RA"]},{row["DEC"]},50") # color=red\n')

In [7]:
cat_src = cat[distance < 500/3600 * u.deg]

with open('../Data/Spectrum/src_deg.reg', 'w') as f:
    # f.write('fk5\n')
    f.write(f'circle({coord_obj.ra.deg},{coord_obj.dec.deg},500") # color=white\n')
    for row in cat_src:
        f.write(f'-circle({row["RA"]},{row["DEC"]},50") # color=red\n')

In [8]:
cat_bg = cat[(distance >= 1000/3600 * u.deg) & (distance < 2500/3600 * u.deg)]

with open('../Data/Spectrum/bg_deg.reg', 'w') as f:
    # f.write('fk5\n')
    f.write(f'annulus({coord_obj.ra.deg},{coord_obj.dec.deg},1000", 2500") # color=green\n')
    for row in cat_bg:
        f.write(f'-circle({row["RA"]},{row["DEC"]},50") # color=red\n')

## Generate spectra, ARFs, and RMFs

In [9]:
def run_srctool(eventfiles, srccoord, prefix, srcreg, extpars=2000, backreg=None, tstep=0.5, xgrid=1, log_file=None):
    subprocess.run(['srctool',
                    f'eventfiles={eventfiles}',
                    f'srccoord={srccoord}',
                    f'prefix={prefix}',
                    'todo=SPEC ARF RMF',
                    'insts=1 2 3 4 6',  # read from TM12346 only
                    'writeinsts=8',  # write spectra of individual TMs as well as TM8
                    f'srcreg={srcreg}',
                    f'suffix=_ts{tstep}_xg{xgrid}.fits',
                    f'backreg={backreg}',  # we don't need a spectrum from a background region as we are interested in the astrophysical background ourselves, not sources on top of the background
                    'exttype=TOPHAT',  # assume the source is uniformly bright to estimate the effective area
                    f'extpars={extpars}',  # approximate diameter of the tophat circle that will cover the full spectral extraction region (in arcsec)
                    f'tstep={tstep}',  # time step for the light curve
                    f'xgrid={xgrid}',  # these numbers are the trade-off between accuracy and time.
                    'psftype=NONE',  # do not correct for psf
                    'pat_sel=15',  # all valid patterns
                    'clobber=yes'
                    ],
                   stdout=log_file,
                   stderr=log_file)

In [10]:
with open('../Data/Spectrum/Spec_SRC_log.txt', 'w+') as log_file:

    run_srctool('../Data/Filtered_data/Merged/Merged_020_s05_TM0_Events.fits', 
                'icrs; 83.63240,+22.01740',
                '../Data/Spectrum/Spec_Src_', 
                '../Data/Spectrum/src_deg.reg', 
                # '../Data/Spectrum/bg_deg.reg', 
                log_file=log_file)
    
    log_file.seek(0)
    log_content = log_file.readlines()
    srctool_count = sum(1 for line in log_content if 'srctool: DONE' in line)
    if srctool_count == 1:
        print('srctool tasks completed successfully')

srctool tasks completed successfully


In [20]:
with open('../Data/Spectrum/Spec_BG_log.txt', 'w+') as log_file:

    run_srctool('../Data/Filtered_data/Merged/Merged_020_s05_TM0_Events.fits', 
                'icrs; 83.63240,+22.01740',
                '../Data/Spectrum/Spec_BG_', 
                # '../Data/Spectrum/src_deg.reg', 
                '../Data/Spectrum/bg_deg.reg', 
                tstep=2,
                xgrid=2,
                log_file=log_file)
    
    log_file.seek(0)
    log_content = log_file.readlines()
    srctool_count = sum(1 for line in log_content if 'srctool: DONE' in line)
    if srctool_count == 1:
        print('srctool tasks completed successfully')

srctool tasks completed successfully


In [21]:
with open('../Data/Spectrum/SRC_and_BG/Spec_SRC_BG_log.txt', 'w+') as log_file:

    run_srctool('../Data/Filtered_data/Merged/Merged_020_s05_TM0_Events.fits', 
                'icrs; 83.63240,+22.01740',
                '../Data/Spectrum/SRC_and_BG/Spec_SRC_', 
                '../Data/Spectrum/src_deg.reg', 
                '../Data/Spectrum/bg_deg.reg', 
                tstep=0.5,
                xgrid=1,
                log_file=log_file)
    
    log_file.seek(0)
    log_content = log_file.readlines()
    srctool_count = sum(1 for line in log_content if 'srctool: DONE' in line)
    if srctool_count == 1:
        print('srctool tasks completed successfully')

srctool tasks completed successfully


### Group the spectra

In [16]:
def run_ftgrouppha(directory, infile, respfile):
    outfile = infile.replace('.fits', '.grp')
    subprocess.run(["ftgrouppha",
                    f"infile={directory}{infile}",
                    f"outfile={directory}{outfile}",
                    "grouptype=opt",
                    f"respfile={directory}{respfile}"])

In [18]:
run_ftgrouppha('../Data/Spectrum/SRC_and_BG/', 
               'Spec_SRC_820_SourceSpec_00001_ts0.5_xg1.fits', 
               'Spec_SRC_820_RMF_00001_ts0.5_xg1.fits')

### Corrections

In [22]:
exposure_src = fits.getval("../Data/Spectrum/Spec_Src_820_SourceSpec_00001_ts0.5_xg1.fits", 
                           "exposure", ext=1)
exposure_bkg = fits.getval("../Data/Spectrum/Spec_BG_820_SourceSpec_00001_ts2_xg2.fits", 
                           "exposure", ext=1)
backscal_src = fits.getval("../Data/Spectrum/Spec_Src_820_SourceSpec_00001_ts0.5_xg1.fits", 
                           "backscal", ext=1)
backscal_bkg = fits.getval("../Data/Spectrum/Spec_BG_820_SourceSpec_00001_ts2_xg2.fits", 
                           "backscal", ext=1)
regarea_src = fits.getval("../Data/Spectrum/Spec_Src_820_SourceSpec_00001_ts0.5_xg1.fits", 
                          "regarea", ext=1)
regarea_bkg = fits.getval("../Data/Spectrum/Spec_BG_820_SourceSpec_00001_ts2_xg2.fits", 
                          "regarea", ext=1)
arfmax_src = fits.getdata("../Data/Spectrum/Spec_Src_820_ARF_00001_ts0.5_xg1.fits", 
                          ext=1)["SPECRESP"].max()
arfmax_bkg = fits.getdata("../Data/Spectrum/Spec_BG_820_ARF_00001_ts2_xg2.fits", 
                          ext=1)["SPECRESP"].max()

r_src = regarea_src / backscal_src
r_bkg = regarea_bkg / backscal_bkg


# copy files and modify values
shutil.copy("../Data/Spectrum/Spec_Src_820_SourceSpec_00001_ts0.5_xg1.fits",
            "../Data/Spectrum/Spec_Src_820_SourceSpec_00001_ts0.5_xg1_corr.fits")
fits.setval("../Data/Spectrum/Spec_Src_820_SourceSpec_00001_ts0.5_xg1_corr.fits",
            "exposure", value=exposure_src / r_src, ext=1)
shutil.copy("../Data/Spectrum/Spec_BG_820_SourceSpec_00001_ts2_xg2.fits",
            "../Data/Spectrum/Spec_BG_820_SourceSpec_00001_ts2_xg2_corr.fits")
fits.setval("../Data/Spectrum/Spec_BG_820_SourceSpec_00001_ts2_xg2_corr.fits",
            "exposure", value=exposure_bkg / r_bkg * (regarea_bkg / regarea_src) , ext=1)

shutil.copy("../Data/Spectrum/Spec_Src_820_ARF_00001_ts0.5_xg1.fits",
            "../Data/Spectrum/Spec_Src_820_ARF_00001_ts0.5_xg1_corr.fits",)
with fits.open("../Data/Spectrum/Spec_Src_820_ARF_00001_ts0.5_xg1_corr.fits") as hdu:
    hdu[1].data["SPECRESP"] *= r_src
    hdu.writeto("../Data/Spectrum/Spec_Src_820_ARF_00001_ts0.5_xg1_corr.fits", overwrite=True)
shutil.copy("../Data/Spectrum/Spec_BG_820_ARF_00001_ts2_xg2.fits",
            "../Data/Spectrum/Spec_BG_820_ARF_00001_ts2_xg2_corr.fits",)
with fits.open("../Data/Spectrum/Spec_BG_820_ARF_00001_ts2_xg2_corr.fits") as hdu:
    hdu[1].data["SPECRESP"] *= r_bkg
    hdu.writeto("../Data/Spectrum/Spec_BG_820_ARF_00001_ts2_xg2_corr.fits", overwrite=True)

In [23]:
specdata_src = fits.getdata("../Data/Spectrum/Spec_Src_820_SourceSpec_00001_ts0.5_xg1_corr.fits",
                            ext=1)["COUNTS"]
specdata_bkg = fits.getdata("../Data/Spectrum/Spec_BG_820_SourceSpec_00001_ts2_xg2_corr.fits",
                            ext=1)["COUNTS"]
channel_elow = fits.getdata("../Data/Spectrum/Spec_Src_820_RMF_00001_ts0.5_xg1.fits", 
                            ext=2)["E_MIN"]  # Channel energy bounds are stored in RMF files
exp_src = fits.getval("../Data/Spectrum/Spec_Src_820_SourceSpec_00001_ts0.5_xg1_corr.fits", 
                      "exposure", ext=1)
exp_bkg = fits.getval("../Data/Spectrum/Spec_BG_820_SourceSpec_00001_ts2_xg2_corr.fits", 
                      "exposure", ext=1)
backscal_src = fits.getval("../Data/Spectrum/Spec_Src_820_SourceSpec_00001_ts0.5_xg1_corr.fits", 
                           "backscal", ext=1)
backscal_bkg = fits.getval("../Data/Spectrum/Spec_BG_820_SourceSpec_00001_ts2_xg2_corr.fits", 
                           "backscal", ext=1)

channel_sel = (channel_elow > 6) & (channel_elow < 9)
areascal_ratio = (specdata_src[channel_sel].sum() / exp_src) / \
                 (specdata_bkg[channel_sel].sum() / exp_bkg) / \
                 (backscal_src / backscal_bkg)
areascal_ratio_err = areascal_ratio * (specdata_src[channel_sel].sum() ** -1. + specdata_bkg[channel_sel].sum()**-1.)**0.5
display(f"{areascal_ratio:.3f}+/-{areascal_ratio_err:.3f}")

'122.944+/-9.507'